# Spatial visualization of MESH runs

This notebook is used to spatially visualize results from MESH runs. It is used to map the rank in the results netcdf of the simulation results with the corresponding polygon within the geofabric shapefile. This allows a seamless visualization using geopandas. 

The results from geofabric, i.e. catchment and river shapefiles, must be located in ./0-geofabric/results within the repository. Alternatively it can provide the path to the geofabric saved by default in the scracth directory. 

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import geopandas as gpd
import math


In [ ]:
drainage = xr.open_dataset('./MESH_drainage_database.nc')
forcing = xr.open_dataset('./MESH_forcing.nc')
catchment = gpd.read_file('../0-geofabric/results/4815_catchments.shp')

# results to be plotted
results = xr.open_dataset('./results/QO_H_GRD.nc')
plot_var = 'QO'
plot_time = 0


In [ ]:
# we can check that the order of the subbasins is the same for both datasets
drainage.load()
forcing.load()


In [ ]:
# obtain the order of the subbasin from the forcing file (this was created in 2-model-specific). 
# note: this can also be done from the drainage file since they both have the same order
order = forcing['subbasin'].to_numpy()

# reorder the geofabric catchment file based on the forcing file
catchment.set_index('COMID', inplace=True)
catchment = catchment.reindex(order)

# adds the rank to the geofabric catchment file
catchment['Rank'] = drainage['Rank']

# adds the plot var as a new column to plot results in catchment
catchment['plot_var'] = results[plot_var].isel(time=plot_time,lat=0)
catchment = catchment.reset_index()


In [ ]:
# plotting 
var = catchment['plot_var']

vmin = math.floor(min(var.min().values,var.min()))
vmax = math.floor(min(var.max().values,var.max()))

fig, ax = plt.subplots(layout='constrained')
sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbr = fig.colorbar(sm, ax=ax)

catchment.plot(ax=ax,column='plot_var', vmin=vmin,vmax=vmax, edgecolor="face");

# adjust the title according to the plotted variable
plt.title('Streamflow data')

In [ ]:
fig,ax = plt.subplots(figsize=(20, 20))
catchment.plot(color='white', edgecolor='k', label = 'Catchment Boundary', linewidth=0.1, ax = ax)
river.plot(color='blue', label = 'River Network', linewidth=0.3, ax=ax)